In [1]:
from bs4 import BeautifulSoup
from random import randint, shuffle
from time import sleep
import re
import pandas as pd
import json
import math
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
from tqdm import tqdm_notebook as tqdm
import shutil
import numpy as np
import pandas_profiling
import datetime

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
sns.set_context("paper", font_scale=1.8)
sns.set_palette("GnBu_d")

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)
pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option("display.max_info_rows", 200)

In [2]:
import sephora_scrape as scrape
%reload_ext autoreload
%autoreload 2

In [ ]:
import pymongo

myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
db = myclient["Sephora"]
c = db["Sephora_Reviews"]

In [ ]:
reviews_folder = "/Users/valmadrid/DataScienceBootcamp/Projects/Gift Recommendation/Gift-Recommendation/reviews/"
if not os.path.exists(reviews_folder):
    print("check file path")

In [ ]:
downloaded_json_reviews = [
    file for file in os.listdir(reviews_folder)
    if (not file.endswith("_stats.json")) and (file.endswith(".json"))
]
downloaded_json_reviews.sort()
len(downloaded_json_reviews), downloaded_json_reviews[:10]

In [ ]:
class MongoHandler():
    
    def __init__(self, row_dict = None, collection = None):
        myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
        db = myclient["Sephora"] 
        self.table = row_dict
        
        c = db[collection]
        c.insert_many(self.table)

In [ ]:
no_reviews = []

for filename in tqdm(downloaded_json_reviews):
    product_id = filename.split("_")[0]

    with open(reviews_folder + filename) as json_file:
        reviews_data = json.load(json_file)

    if reviews_data["TotalResults"] == 0 or reviews_data["HasErrors"] == True:
        no_reviews.append(filename)
        continue

    else:
        review_list = reviews_data["Results"]
        for rev in review_list:
            rev["product_id"] = product_id
        MongoHandler(review_list, "Sephora_Reviews")
#         df_reviews = df_reviews.append(pd.DataFrame(review_list_cleaned),
#                                                    ignore_index=True)
#         review_list, review_list_cleaned = [], []
#         df_reviews.to_csv("test_all_reviews.csv", index=False)
    reviews_data = {}

In [ ]:
for i in range(1,6):
    cursor = c.find({"Rating": {"$gte": i}})
    reviews = pd.DataFrame(cursor)
    reviews.to_csv("all_reviews_"+str(i)+".csv")
    reviews = pd.DataFrame()

In [ ]:
reviews = pd.DataFrame()

In [ ]:
reviews_r = pd.read_csv("all_reviews_3.csv")
reviews_r.SubmissionTime = pd.to_datetime(reviews_r.SubmissionTime,
                                          format="%Y-%m-%d")
reviews = reviews.append(
    reviews_r[(reviews_r.SubmissionTime > datetime.strptime(
        "2018-06-25 23:59:59+0000", "%Y-%m-%d %H:%M:%S%z"))
              & (reviews_r.IsRecommended == True)].copy(),
    ignore_index=True,
    sort=False)
reviews
del reviews_r
reviews.shape

In [ ]:
reviews_r = pd.read_csv("all_reviews_4.csv")
reviews_r.SubmissionTime = pd.to_datetime(reviews_r.SubmissionTime,
                                          format="%Y-%m-%d")
reviews = reviews.append(
    reviews_r[(reviews_r.SubmissionTime > datetime.strptime(
        "2018-06-25 23:59:59+0000", "%Y-%m-%d %H:%M:%S%z"))
              & (reviews_r.IsRecommended == True)].copy(),
    ignore_index=True,
    sort=False)
reviews
del reviews_r
reviews.shape

In [ ]:
reviews_r = pd.read_csv("all_reviews_5.csv")
reviews_r.SubmissionTime = pd.to_datetime(reviews_r.SubmissionTime,
                                          format="%Y-%m-%d")
reviews = reviews.append(
    reviews_r[(reviews_r.SubmissionTime > datetime.strptime(
        "2018-06-25 23:59:59+0000", "%Y-%m-%d %H:%M:%S%z"))
              & (reviews_r.IsRecommended == True)].copy(),
    ignore_index=True,
    sort=False)
reviews
del reviews_r
reviews.shape

In [ ]:
reviews.columns

In [ ]:
reviews.iloc[0].T

In [ ]:
reviews[reviews.duplicated(subset=["AuthorId", "product_id", "ProductId"],
                           keep=False)].sort_values(
                               ["AuthorId", "product_id", "SubmissionTime"],
                               ascending=False)[[
                                   "AuthorId","UserNickname", "product_id","ProductId", "SubmissionTime",
                                   "Rating", "Id", "ReviewText", "_id"
                               ]]

In [ ]:
reviews.sort_values(["AuthorId", "product_id", "SubmissionTime"],
                               ascending=False).reset_index(drop=True, inplace=True)

In [ ]:
reviews.drop_duplicates(subset=["AuthorId", "product_id"], keep="first", inplace=True)
reviews.shape

In [ ]:
reviews.to_csv("selected_reviews.csv", index=False)

In [2]:
reviews = pd.read_csv("selected_reviews.csv")
reviews.head()

/Applications/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,_id,UserNickname,Rating,ReviewText,ContextDataValues,ProductId,UserEmailAddress,Videos,CommentIds,Photos,TotalNegativeFeedbackCount,RatingRange,CampaignId,UserLocation,SecondaryRatingsOrder,TotalCommentCount,ContextDataValuesOrder,ClientResponses,TagDimensionsOrder,TotalPositiveFeedbackCount,IsRatingsOnly,LastModificationTime,Pros,ProductRecommendationIds,BadgesOrder,AuthorId,Helpfulness,SubmissionTime,AdditionalFields,Title,AdditionalFieldsOrder,TotalFeedbackCount,ModerationStatus,IsFeatured,Cons,TagDimensions,SecondaryRatings,Badges,SubmissionId,IsSyndicated,IsRecommended,LastModeratedTime,Id,ContentLocale,product_id
0,5ef5789b0642bf88095d59b4,alycat88,3,"It doesn't help with blackheads at all, but it does with whiteheads. It is okay","{'skinType': {'Value': 'combination', 'Id': 'skinType', 'ValueLabel': 'Combination', 'DimensionLabel': 'Skin Type'}, 'eyeColor': {'Value': 'green', 'Id': 'eyeColor', 'ValueLabel': 'Green', 'DimensionLabel': 'Eye Color'}, 'skinConcerns': {'Value': 'acne', 'Id': 'skinConcerns', 'ValueLabel': 'Acne', 'DimensionLabel': 'Skin concerns'}, 'StaffContext': {'Value': 'false', 'Id': 'StaffContext', 'ValueLabel': 'No', 'DimensionLabel': 'I am a Sephora employee'}, 'hairColor': {'Value': 'brunette', 'Id': 'hairColor', 'ValueLabel': 'Brunette', 'DimensionLabel': 'Hair color'}, 'skinTone': {'Value': 'light', 'Id': 'skinTone', 'ValueLabel': 'Light', 'DimensionLabel': 'Skin Tone'}, 'IncentivizedReview': {'Value': 'false', 'Id': 'IncentivizedReview', 'ValueLabel': 'False', 'DimensionLabel': 'I received this product as a free sample'}}",1408400,nan,[],[],[],1,5,nan,nan,[],0,"['eyeColor', 'hairColor', 'skinTone', 'skinType', 'skinConcerns', 'IncentivizedReview', 'StaffContext']",[],[],2,False,2020-05-29T03:13:31.000+00:00,nan,[],[],5869746961,0.67,2019-09-14 17:16:59+00:00,"{'sociallockup': {'Label': 'sociallockup', 'Value': 'avatar=https://sephora.i.lithium.com/t5/image/serverpage/avatar-name/default-avatar/avatar-theme/sephora/avatar-collection/sephora/avatar-display-size/profile/version/2?xdesc=1.0|biBadgeUrl=/html/rank_icons/birole_vib.svg|engagementBadgeUrl=/html/rank_icons/rank_rookie-01.svg|biTier=VIB', 'Id': 'sociallockup'}}",NaN,['sociallockup'],3,APPROVED,False,nan,{},{},{},2sta8dmfkuibcayirk1137v8g,False,True,2019-09-14T20:15:05.000+00:00,135835739,en_CA,P0417
1,5ef5789b0642bf88095d59ba,divinebelle,3,"Be careful using this, it does extract pimples but only if they are at a head and ready to pop. Otherwise the harder you push to bring it out marks your skin. It’s better than using your nails and bruising your skin on stubborn pimples but still can damage your skin. The black head extractor does work fantastic, brings out the really stubborn black heads but then again also damages your skin if your press too hard. Trail and error learning how hard to press before making marks.","{'skinType': {'Value': 'combination', 'Id': 'skinType', 'ValueLabel': 'Combination', 'DimensionLabel': 'Skin Type'}, 'eyeColor': {'Value': 'blue', 'Id': 'eyeColor', 'ValueLabel': 'Blue', 'DimensionLabel': 'Eye Color'}, 'skinConcerns': {'Value': 'acne', 'Id': 'skinConcerns', 'ValueLabel': 'Acne', 'DimensionLabel': 'Skin concerns'}, 'StaffContext': {'Value': 'false', 'Id': 'StaffContext', 'ValueLabel': 'No', 'DimensionLabel': 'I am a Sephora employee'}, 'hairColor': {'Value': 'black', 'Id': 'hairColor', 'ValueLabel': 'Black', 'DimensionLabel': 'Hair color'}, 'skinTone': {'Value': 'medium', 'Id': 'skinTone', 'ValueLabel': 'Medium', 'DimensionLabel': 'Skin Tone'}, 'IncentivizedReview': {'Value': 'false', 'Id': 'IncentivizedReview', 'ValueLabel': 'False', 'DimensionLabel': 'I received this product as a free sample'}}",1408400,nan,[],[],[],0,5,nan,nan,[],0,"['eyeColor', 'hairColor', 'skinTone', 'skinType', 'skinConcerns', 'IncentivizedReview', 'StaffContext']",[],[],5,False,2020-04-30T00:37:29.000+00:00,nan,[],[],22189528930,1.00,2019-07-07 17:48:13+00:00,"{'sociallockup': {'Label': 'sociallockup', 'Value': 'avatar=https://community.

In [3]:
reviews.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 939049 entries, 0 to 939048
Data columns (total 45 columns):
_id                           939049 non-null object
UserNickname                  933862 non-null object
Rating                        939049 non-null int64
ReviewText                    933305 non-null object
ContextDataValues             939049 non-null object
ProductId                     939049 non-null object
UserEmailAddress              0 non-null float64
Videos                        939049 non-null object
CommentIds                    939049 non-null object
Photos                        939049 non-null object
TotalNegativeFeedbackCount    939049 non-null int64
RatingRange                   939049 non-null int64
CampaignId                    0 non-null float64
UserLocation                  0 non-null float64
SecondaryRatingsOrder         939049 non-null object
TotalCommentCount             939049 non-null int64
ContextDataValuesOrder        939049 non-null object
Clie

In [11]:
# ratings = pd.DataFrame(
#     data=reviews.groupby("product_id")["Rating"].mean().reset_index().values,
#     columns=["id", "ave_rating"])
# ratings["review_count"] = reviews.groupby(
#     "product_id")["Rating"].count().values
# ratings.head()

In [12]:
# plt.figure(figsize=(15,10))
# sns.jointplot("ave_rating", "review_count", data=ratings)
# plt.show()

In [4]:
reviews_short = reviews[[
    "product_id", "AuthorId", "SubmissionTime", "Id", "Rating", "Helpfulness",
    "TotalFeedbackCount", "TotalPositiveFeedbackCount",
    "TotalNegativeFeedbackCount", "TotalCommentCount", "Title", "ReviewText", "ContextDataValues"
]].copy()
reviews_short.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 939049 entries, 0 to 939048
Data columns (total 13 columns):
product_id                    939049 non-null object
AuthorId                      939049 non-null object
SubmissionTime                939049 non-null object
Id                            939049 non-null int64
Rating                        939049 non-null int64
Helpfulness                   312655 non-null float64
TotalFeedbackCount            939049 non-null int64
TotalPositiveFeedbackCount    939049 non-null int64
TotalNegativeFeedbackCount    939049 non-null int64
TotalCommentCount             939049 non-null int64
Title                         626091 non-null object
ReviewText                    933305 non-null object
ContextDataValues             939049 non-null object
dtypes: float64(1), int64(6), object(6)
memory usage: 93.1+ MB


In [5]:
del reviews

In [6]:
reviews_short.rename(
    {
        "AuthorId": "author_id",
        "SubmissionTime": "date",
        "Id": "review_id",
        "Rating": "rating",
        "Helpfulness": "helpfulness",
        "TotalFeedbackCount": "feedback_count",
        "TotalPositiveFeedbackCount": "positive_feedback",
        "TotalNegativeFeedbackCount": "negative_feedback",
        "TotalCommentCount": "comment_count",
        "ContextDataValues": "context",
        "Title": "review_title",
        "ReviewText": "review_text"
    },
    axis=1,
    inplace=True)

reviews_short.head()

,product_id,author_id,date,review_id,rating,helpfulness,feedback_count,positive_feedback,negative_feedback,comment_count,review_title,review_text,context
0,P0417,5869746961,2019-09-14 17:16:59+00:00,135835739,3,0.67,3,2,1,0,NaN,"It doesn't help with blackheads at all, but it does with whiteheads. It is okay","{'skinType': {'Value': 'combination', 'Id': 'skinType', 'ValueLabel': 'Combination', 'DimensionLabel': 'Skin Type'}, 'eyeColor': {'Value': 'green', 'Id': 'eyeColor', 'ValueLabel': 'Green', 'DimensionLabel': 'Eye Color'}, 'skinConcerns': {'Value': 'acne', 'Id': 'skinConcerns', 'ValueLabel': 'Acne', 'DimensionLabel': 'Skin concerns'}, 'StaffContext': {'Value': 'false', 'Id': 'StaffContext', 'ValueLabel': 'No', 'DimensionLabel': 'I am a Sephora employee'}, 'hairColor': {'Value': 'brunette', 'Id': 'hairColor', 'ValueLabel': 'Brunette', 'DimensionLabel': 'Hair color'}, 'skinTone': {'Value': 'light', 'Id': 'skinTone', 'ValueLabel': 'Light', 'DimensionLabel': 'Skin Tone'}, 'IncentivizedReview': {'Value': 'false', 'Id': 'IncentivizedReview', 'ValueLabel': 'False', 'DimensionLabel': 'I received this product as a free sample'}}"
1,P0417,22189528930,2019-07-07 17:48:13+00:00,130283148,3,1.00,5,5,0,0,NaN,"Be careful using this, it does extract pimples but only if they are at a head and ready to pop. Otherwise the harder you push to bring it out marks your skin. It’s better than using your nails and bruising your skin on stubborn pimples but still can damage your skin. The black head extractor does work fantastic, brings out the really stubborn black heads but then again also damages your skin if your press too hard. Trail and error learning how hard to press before making marks.","{'skinType': {'Value': 'combination', 'Id': 'skinType', 'ValueLabel': 'Combination', 'DimensionLabel': 'Skin Type'}, 'eyeColor': {'Value': 'blue', 'Id': 'eyeColor', 'ValueLabel': 'Blue', 'DimensionLabel': 'Eye Color'}, 'skinConcerns': {'Value': 'acne', 'Id': 'skinConcerns', 'ValueLabel': 'Acne', 'DimensionLabel': 'Skin concerns'}, 'StaffContext': {'Value': 'false', 'Id': 'StaffContext', 'ValueLabel': 'No', 'DimensionLabel': 'I am a Sephora employee'}, 'hairColor': {'Value': 'black', 'Id': 'hairColor', 'ValueLabel': 'Black', 'DimensionLabel': 'Hair color'}, 'skinTone': {'Value': 'medium', 'Id': 'skinTone', 'ValueLabel': 'Medium', 'DimensionLabel': 'Skin Tone'}, 'IncentivizedReview': {'Value': 'false', 'Id': 'IncentivizedReview', 'ValueLabel': 'False', 'DimensionLabel': 'I received this product as a free sample'}}"
2,P0417,5641704837,2018-09-15 16:34:33+00:00,118730320,3,nan,0,0,0,0,NaN,It works but I think it’s not as gentle as I would like. It can still cause trauma on your skin. So be gentle! Better than using your fingers.,"{'skinType': {'Value': 'combination', 'Id': 'skinType', 'ValueLabel': 'Combination', 'DimensionLabel': 'Skin Type'}, 'eyeColor': {'Value': 'blue', 'Id': 'eyeColor', 'ValueLabel': 'Blue', 'DimensionLabel': 'Eye Color'}, 'skinConcerns': {'Value': 'acne', 'Id': 'skinConcerns', 'ValueLabel': 'Acne', 'DimensionLabel': 'Skin concerns'}, 'hairColor': {'Value': 'red', 'Id': 'hairColor', 'ValueLabel': 'Red', 'DimensionLabel': 'Hair color'}, 'skinTone': {'Value': 'fair', 'Id': 'skinTone', 'ValueLabel': 'Fair', 'DimensionLabel': 'Skin Tone'}}"
3,P04546871,22081371793,2020-05-17 18:17:18+00:00,156485522,3,1.00,3,3,0,0,"She’s cute, glitter will get on your clothing",Bought this on sale and used it for a shoot at home. It’s super cute and feels very nice with amazing sparkle pay off in the light. Highly recommend using a body brush for this just to avoid it being clustered in your hands. The glitter will get on everything you touch your skin on as a heads up. Great for warm outings to amp your look.,"{'skinType': {'Value': 'combination', 'Id': 'skinType', 'ValueLabel': 'Combination', 'DimensionLabel': 'Skin Type'}, 'eyeColor': {'Value': 'brown', 'Id': 'eyeColor', 'ValueLabel': 'Brown', 'DimensionLabel': 'Eye Color'}, 'skinConcerns': {'Value': 'acne'

In [7]:
for i in tqdm(range(len(reviews_short))):

    if reviews_short["context"].iloc[i]:
        context = json.loads(reviews_short["context"].iloc[i].replace("\'", "\""))
        if context=={}:
            pass
        else:
            for context, value in context.items():
                for context2, value2 in value.items():
                    col_name = (re.sub(r"[^a-zA-Z0-9]+", " ",
                                       context.lower())).replace(" ", "_")
                    reviews_short.at[i, col_name] = value["Value"]

#     if reviews_short["tags"].iloc[i]:
#         tags = json.loads(reviews_short["tags"].iloc[i].replace("\'", "\""))
#         if tags=={}:
#             pass
#         else:
#             reviews_short.at[i, "tags_list"] = tags["Pro"]["Values"]

In [8]:
reviews_short.drop(columns="context", inplace=True)

In [9]:
reviews_short.isna().sum()

product_id            0     
author_id             0     
date                  0     
review_id             0     
rating                0     
helpfulness           626394
feedback_count        0     
positive_feedback     0     
negative_feedback     0     
comment_count         0     
review_title          312958
review_text           5744  
skintype              11647 
eyecolor              28310 
skinconcerns          391404
staffcontext          284934
haircolor             17844 
skintone              11748 
incentivizedreview    284934
verifiedpurchaser     692532
age                   838900
hairconcerns          846079
haircondition         907437
dtype: int64

In [10]:
reviews_short.helpfulness.fillna(2, inplace=True)

In [12]:
reviews_short.to_csv("selected_reviews_short.csv", index=False)